#### Milestone II
#### Michael Perrine
#### DSC 540
#### Professor Williams


# <h1><center>Market Analysis</center></h1>


The purpose of this analysis is to research market data. I will gather data from several sources. The goal is to analyze the effect of economic data on stock performance. I will also use the S&P 500 as the benchmark for my stock performance. The sources of my data will come from various sources. I will pull the stock data from FMP a stock price API. The web address for FMP is Documentation V2 - API Reference | FMP.  The economic data will come from the FRED. The FRED is managed by the St. louis Fed. The web address is Federal Reserve Economic Data | FRED | St. Louis Fed. Finally,  I will pull S&P data from Wikipedia. The web address is S&P 500 - Wikipedia.  Economic data will show a relationship between the stock data and the economy. And the S&P will show the relationship between the stock index and the market. 
	
The plan for my analysis will be to add the different data sets into one data frame. I will run an exploratory analysis on the data. I will need to transform the data and determine the variables that make the greatest impact on the analysis. It will be important for me to build graphs to determine the connection between the variables. After I build my charts, I can  run a regression analysis and test my model. For the regression analysis I will need to split my data into a training and a testing set. Along with any analysis there are challenges that I will have to manage. 

Some challenges that I can foresee are issues with combining the different data sets. Since I am working with data sets from multiple sources, I’ll have to make sure they are formatted appropriately. There may also be problems with loading the data into the program. The data will be stored in different formats, so I’ll have to ensure that they are formatted in a manner that is compatible across data sets. Another challenge that I can anticipate is overfitting the data. I need to make that the training and testing sets are not in danger of overfitting the data.

Ethical concerns are always a factor when manipulating data. It is important to implement safeguards to prevent ethical. One ethical concern is the  misrepresentation of the data. It is important that I don’t allow the data to misrepresent the results. Another ethical issue is bias. It’s important that I don’t allow my personal bias to cause me to see relationships that don’t exist. 
	


The first series of code will load all the libraries for the analysis

In [1]:
# Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import yfinance as yf
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests 

1. Prep the flat files


The next series of code loads the data from the FRED. I will use three data sets for my analysis.
The first data set is the Coincident Economic Activity Index. This index measures the US economic condition. It takes into account non-farm payroll employment, the unemployment rate, average hours worked in manufacturing, and wages and salaries. The second data set is the Consumer Price Index. This measures the price changes of goods and services and is a leading indicator of inflation. The final data set from the FRED is the One Year T-Bill. Fluctuations in T-Bill rates indicate direction of the US Monetary Policy.

In [2]:
# This code loads in the cea data, creates a pandas dataframe, and displays the first 5 rows
cea = pd.read_csv(r"coincident economic activity index.csv")
cea = pd.DataFrame(cea)
cea.head()

,observation_date,USPHCI
0,1/1/1979,45.36
1,2/1/1979,45.51
2,3/1/1979,45.75
3,4/1/1979,45.82
4,5/1/1979,46.05


In [3]:
# This code loads in the cpi data , creates a pandas dataframe, and displays the first 5 rows
cpi = pd.read_csv(r"monthly cpi.csv")
cpi = pd.DataFrame(cpi)
cpi.head()

,observation_date,CPIAUCSL
0,1/1/1947,21.48
1,2/1/1947,21.62
2,3/1/1947,22.00
3,4/1/1947,22.00
4,5/1/1947,21.95


In [4]:
# This code loads in the t-bill data , creates a pandas dataframe,and displays the first 5 rows
t_bill = pd.read_csv(r"one year t-bill.csv")
t_bill = pd.DataFrame(t_bill)
t_bill.head()

,observation_date,TB1YR
0,7/1/1959,4.34
1,8/1/1959,4.31
2,9/1/1959,4.83
3,10/1/1959,4.69
4,11/1/1959,4.54


The next step is to combine the three data frames into one. This will give me the opportunity to do further analysis. The following series of codes will prep and combine the three data frames.

In [5]:
# The first line of code will store all three data frames into one object
dfs = (cea, cpi, t_bill)

In [6]:
# The second line of code combines all the data into one new data frame.
new_data = reduce(lambda left, right: pd.merge(left, right, on = "observation_date", how = "outer"),dfs)


In [7]:
# The third line of code renames the columns in the data frame and displays the completed data frame
new_data = new_data.set_axis(["date", "cea", "cpi" , "t_bill"], axis=1)
new_data

,date,cea,cpi,t_bill
0,1/1/1947,NaN,21.480,NaN
1,1/1/1948,NaN,23.680,NaN
2,1/1/1949,NaN,24.010,NaN
3,1/1/1950,NaN,23.510,NaN
4,1/1/1951,NaN,25.380,NaN
...,...,...,...,...
934,9/1/2020,122.45,259.997,0.13
935,9/1/2021,130.21,273.942,0.07
936,9/1/2022,136.71,296.421,3.73
937,9/1/2023,140.69,307.374,5.15


Now that I have one data frame I want to reverse the order of the elements. Currently they are in ascending order. I want them in descending order that way when I add my stock data I'll see everything from the most current date to the oldest date.

In [8]:
# This line of code reverses the order of the elements
new_data = new_data.iloc[::-1].reset_index(drop=True)
new_data


,date,cea,cpi,t_bill
0,9/1/2024,144.20,314.851,3.87
1,9/1/2023,140.69,307.374,5.15
2,9/1/2022,136.71,296.421,3.73
3,9/1/2021,130.21,273.942,0.07
4,9/1/2020,122.45,259.997,0.13
...,...,...,...,...
934,1/1/1951,NaN,25.380,NaN
935,1/1/1950,NaN,23.510,NaN
936,1/1/1949,NaN,24.010,NaN
937,1/1/1948,NaN,23.680,NaN


Now I want to view the data types in my data frame. It's important to make sure all the data types are appropriate for the elements. It looks like I need to change the date data type. It's listed as an object and should be listed as date/time.

In [9]:
# This code displays the data type
new_data.dtypes

date       object
cea       float64
cpi       float64
t_bill    float64
dtype: object

In [10]:
new_data.head()

,date,cea,cpi,t_bill
0,9/1/2024,144.20,314.851,3.87
1,9/1/2023,140.69,307.374,5.15
2,9/1/2022,136.71,296.421,3.73
3,9/1/2021,130.21,273.942,0.07
4,9/1/2020,122.45,259.997,0.13


In [11]:
# This code changes the date data type from object to date/time and displays the result
new_data["date"] = pd.to_datetime(new_data["date"],format='mixed')
new_data.dtypes

date      datetime64[ns]
cea              float64
cpi              float64
t_bill           float64
dtype: object

In [12]:
# This code displays the new date format
new_data.head()

,date,cea,cpi,t_bill
0,2024-09-01,144.20,314.851,3.87
1,2023-09-01,140.69,307.374,5.15
2,2022-09-01,136.71,296.421,3.73
3,2021-09-01,130.21,273.942,0.07
4,2020-09-01,122.45,259.997,0.13


In [13]:
# This code drops NaN values. I chose to drop the 
new_data_1 = new_data.dropna(how="any")
new_data_1

,date,cea,cpi,t_bill
0,2024-09-01,144.20,314.851,3.87
1,2023-09-01,140.69,307.374,5.15
2,2022-09-01,136.71,296.421,3.73
3,2021-09-01,130.21,273.942,0.07
4,2020-09-01,122.45,259.997,0.13
...,...,...,...,...
902,1983-01-01,47.12,97.900,8.01
903,1982-01-01,47.54,94.400,12.77
904,1981-01-01,47.16,87.200,12.62
905,1980-01-01,46.74,78.000,10.96


In [14]:
# This code validates the dimensions of the cleaned data frame
new_data_1.shape

(475, 4)

In [28]:
url = 'https://en.wikipedia.org/wiki/S&P_500'

In [29]:
page = requests.get(url)

page

soup = BeautifulSoup(page.text, 'html.parser')


In [31]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>S&amp;P 500 - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-widt

In [48]:
data = soup.find_all('table')[1]
data

<table class="wikitable mw-collapsible sortable" style="text-align:right;">
<caption class="nowrap">Show / Hide table
</caption>
<tbody><tr>
<th rowspan="2">Year
</th>
<th rowspan="2">Change in<br/>Index
</th>
<th rowspan="2">Total<br/>Annual Return,<br/>including<br/>dividends
</th>
<th rowspan="2">Value of $1.00<br/>invested on<br/>January 1, 1970
</th>
<th colspan="5">Annualized Return over
</th></tr>
<tr>
<th>5 years
</th>
<th>10 years
</th>
<th>15 years
</th>
<th>20 years
</th>
<th>25 years
</th></tr>
<tr>
<td>1961
</td>
<td>23.13%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td></tr>
<tr>
<td>1962
</td>
<td style="color:red">-11.81%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td></tr>
<tr>
<td>1963
</td>
<td>18.89%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td></tr>
<tr>
<td>1964
</td>
<td>12.97%
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-


In [56]:
data_titles = data.find_all('th')

In [57]:
print(data_titles)

[<th rowspan="2">Year
</th>, <th rowspan="2">Change in<br/>Index
</th>, <th rowspan="2">Total<br/>Annual Return,<br/>including<br/>dividends
</th>, <th rowspan="2">Value of $1.00<br/>invested on<br/>January 1, 1970
</th>, <th colspan="5">Annualized Return over
</th>, <th>5 years
</th>, <th>10 years
</th>, <th>15 years
</th>, <th>20 years
</th>, <th>25 years
</th>, <th>High
</th>, <th>34.11%
</th>, <th>37.58%
</th>, <th>---
</th>, <th>28.56%
</th>, <th>19.21%
</th>, <th>18.93%
</th>, <th>17.88%
</th>, <th>17.25%
</th>, <th>Low
</th>, <th>−38.49%
</th>, <th>−37.00%
</th>, <th>---
</th>, <th>−2.35%
</th>, <th>−1.38%
</th>, <th>4.24%
</th>, <th>5.62%
</th>, <th>7.56%
</th>, <th>Median
</th>, <th>12.40%
</th>, <th>15.79%
</th>, <th>---
</th>, <th>14.09%
</th>, <th>12.75%
</th>, <th>10.76%
</th>, <th>11.25%
</th>, <th>10.26%
</th>, <th rowspan="2">Year
</th>, <th rowspan="2">Change in<br/>Index
</th>, <th rowspan="2">Total<br/>Annual Return,<br/>including<br/>dividends
</th>, <th rowspan="2"

In [58]:
data_title_labels = [title.text for title in data_titles]

In [59]:
print(data_title_labels)

['Year\n', 'Change inIndex\n', 'TotalAnnual Return,includingdividends\n', 'Value of $1.00invested onJanuary 1, 1970\n', 'Annualized Return over\n', '5 years\n', '10 years\n', '15 years\n', '20 years\n', '25 years\n', 'High\n', '34.11%\n', '37.58%\n', '---\n', '28.56%\n', '19.21%\n', '18.93%\n', '17.88%\n', '17.25%\n', 'Low\n', '−38.49%\n', '−37.00%\n', '---\n', '−2.35%\n', '−1.38%\n', '4.24%\n', '5.62%\n', '7.56%\n', 'Median\n', '12.40%\n', '15.79%\n', '---\n', '14.09%\n', '12.75%\n', '10.76%\n', '11.25%\n', '10.26%\n', 'Year\n', 'Change inIndex\n', 'TotalAnnual Return,includingdividends\n', 'Value of $1.00invested onJanuary 1, 1970\n', 'Annualized Return over\n', '5 years\n', '10 years\n', '15 years\n', '20 years\n', '25 years\n']


In [20]:
yf.__version__

'0.2.55'

In [21]:
stock= yf.download(['AAPL', 'MSFT', 'NVDA', 'QQQ'], start= '2020-01-01')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  4 of 4 completed


In [22]:
stock_df = stock
stock_df

Price            Close                                            High  \
Ticker            AAPL        MSFT        NVDA         QQQ        AAPL   
Date                                                                     
2020-01-02   72.716057  153.323288    5.972161  209.325882   72.776583   
2020-01-03   72.009117  151.414093    5.876572  207.408478   72.771745   
2020-01-06   72.582924  151.805481    5.901216  208.744843   72.621661   
2020-01-07   72.241547  150.421387    5.972660  208.715805   72.849224   
2020-01-08   73.403656  152.817368    5.983862  210.284592   73.706287   
...                ...         ...         ...         ...         ...   
2025-04-25  209.279999  391.850006  111.010002  472.559998  209.750000   
2025-04-28  210.139999  391.160004  108.730003  472.410004  211.500000   
2025-04-29  211.210007  394.040009  109.019997  475.529999  212.240005   
2025-04-30  212.500000  395.260010  108.919998  475.470001  213.580002   
2025-05-01  213.320007  425.399994  111.610001  481.679993  214.550003   

Price                                                  Low              \
Ticker            MSFT        NVDA         QQQ        AAPL        MSFT   
Date                                                                     
2020-01-02  153.428291    5.972161  209.325882   71.466797  151.137325   
2020-01-03  152.683675    5.920384  208.657701   71.783962  150.879535   
2020-01-06  151.872308    5.906444  208.773893   70.876090  149.399957   
2020-01-07  152.416438    6.018463  209.306520   72.021231  150.173203   
2020-01-08  153.495120    6.025185  211.243297   71.943766  150.774585   
...                ...         ...         ...         ...         ...   
2025-04-25  392.160004  111.919998  473.089996  206.199997  384.600006   
2025-04-28  392.739990  110.370003  474.809998  207.460007  386.640015   
2025-04-29  395.100006  110.199997  476.420013  208.369995  390.380005   
2025-04-30  396.660004  108.919998  477.200012  206.669998  384.440002   
2025-05-01  436.989990  114.940002  487.179993  208.899994  424.920013   

Price                                     Open                          \
Ticker            NVDA         QQQ        AAPL        MSFT        NVDA   
Date                                                                     
2020-01-02    5.892751  207.214797   71.721004  151.566880    5.943285   
2020-01-03    5.827532  206.536938   71.941328  151.127733    5.852425   
2020-01-06    5.757084  205.529815   71.127881  149.944070    5.783222   
2020-01-07    5.884537  208.057311   72.592594  152.082346    5.929594   
2020-01-08    5.928350  208.357517   71.943766  151.710061    5.968428   
...                ...         ...         ...         ...         ...   
2025-04-25  105.730003  465.390015  206.369995  387.000000  106.849998   
2025-04-28  106.019997  466.029999  210.000000  391.959991  109.690002   
2025-04-29  107.440002  469.589996  208.690002  391.299988  107.669998   
2025-04-30  104.080002  462.429993  209.300003  390.299988  104.470001   
2025-05-01  111.320000  480.739990  209.009995  431.109985  113.099998   

Price                      Volume                                 
Ticker             QQQ       AAPL      MSFT       NVDA       QQQ  
Date                                                              
2020-01-02  207.621517  135480400  22622100  237536000  30969400  
2020-01-03  206.556310  146322800  21116200  205384000  27518900  
2020-01-06  205.781590  118387200  20813700  262636000  21655300  
2020-01-07  208.822328  108872000  21634100  314856000  22139300  
2020-01-08  208.686764  132079200  27746500  277108000  26397300  
...                ...        ...       ...        ...       ...  
2025-04-25  466.970001   38222300  18973200  251064700  38697100  
2025-04-28  473.029999   38743100  16579400  207708500  33550800  
2025-04-29  470.170013   36827600  14974000  170444300  30613100  
2025-04-30  467.130005   52286500  36461100  235044600  46810600  
2025-05-01  483.45

In [23]:
stock_1 = stock_df.unstack
